# TrenchRipper Master Notebook

#### Imports

In [12]:
import trenchripper as tr

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider, Dropdown, IntText, SelectMultiple, IntRangeSlider, FloatRangeSlider
import ipywidgets as widgets
import matplotlib
import warnings
import copy

matplotlib.rcParams['figure.figsize'] = [20, 10]
warnings.filterwarnings(action='once')

In [2]:
from nd2reader import ND2Reader

In [3]:
import nd2reader
dir(nd2reader)
print(nd2reader.__version__)

3.1.0


#### Specify Paths

Begin by defining the directory in which all processing will be done, as well as the initial nd2 file we will be processing.

In [5]:
# headpath = "/n/scratch2/bj66/vibrio_analysis_evaluation"
# nd2file = "/n/scratch2/bj66/vibrio_analysis_evaluation/Vibrio_30_MUX002.nd2"
headpath = "/n/scratch2/bj66/191014/"
nd2file = "/n/scratch2/bj66/191014/noise_suppression_library191014_2.nd2"

#### Transfer files into the scratch folder

In [ ]:
sourcedir = "/n/files/SysBio/PAULSSON\ LAB/Personal\ Folders/Leoncini/DATA_Ti4/20190924--Vibrio30_MUX/test"
targetdir = "/n/scratch2/bj66/vibrio_analysis_evaluation"
tr.cluster.transferjob(sourcedir,targetdir)

### Extract to hdf5 files

#### Start Dask Workers

In [5]:
dask_controller = tr.cluster.dask_controller(walltime='04:00:00',local=False,n_workers=40,memory='2GB')
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

/home/bj66/anaconda3/lib/python3.7/site-packages/bokeh/plotting/helpers.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, OrderedDict, Sequence


In [6]:
dask_controller.displaydashboard()

#### Perform Extraction

In [7]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(nd2file,headpath,tpts_per_file=25)

In [8]:
hdf5_extractor.inter_get_notes()

interactive(children=(Textarea(value='', description='Organism:', placeholder='Organism imaged in this experim…

In [9]:
hdf5_extractor.extract(dask_controller)

/home/bj66/anaconda3/lib/python3.7/site-packages/nd2reader/common_raw_metadata.py:94: RuntimeWarning: Reported average frame interval (56018.0 ms) doesn't match the set interval (60000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)
/home/bj66/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


{'height': 1788, 'width': 2048, 'date': datetime.datetime(2019, 9, 24, 17, 13, 21), 'fields_of_view': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73], 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'z_levels': [], 'total_images_per_channel': 1110, 'channels': ['Phase'], 'pixel_microns': 0.065, 'num_frames': 15, 'experiment': {'description': 'Vibrio_30_MUX', 'loops': [{'start': 0, 'duration': -1.0, 'stimulation': False, 'sampling_interval': 56017.95771428517}]}, 'num_fovs': 74, 'settings': {'Phase': {'camera_name': 'Andor Zyla VSC-04365', 'obj_settings': {'wsObjectiveName': 'Plan Fluor 100x Oil Ph3 ADH', 'dObjectiveMag': 100.0, 'dObjectiveNA': 1.3, 'dRefractIndex': 1.515}, 'Camera_Type': 'Andor_Zyla', 'Binning': '1x1', 'Exposure': '50_ms

/home/bj66/anaconda3/lib/python3.7/site-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'global'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/home/bj66/TrenchRipper/trenchripper/utils.py:91: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.metadata = store.get_storer(key).attrs.metadata


#### Shutdown Dask

In [10]:
dask_controller.shutdown()

Worker tcp://10.120.16.179:60960 restart in Job 52352597. This can be due to memory issue.
Worker tcp://10.120.16.179:56984 restart in Job 52352596. This can be due to memory issue.
Worker tcp://10.120.16.179:42002 restart in Job 52352554. This can be due to memory issue.
Worker tcp://10.120.17.12:46018 restart in Job 52352564. This can be due to memory issue.
Worker tcp://10.120.17.12:45818 restart in Job 52352565. This can be due to memory issue.
Worker tcp://10.120.17.12:55851 restart in Job 52352563. This can be due to memory issue.
Worker tcp://10.120.16.181:37246 restart in Job 52352589. This can be due to memory issue.
Worker tcp://10.120.16.180:36471 restart in Job 52352591. This can be due to memory issue.
Worker tcp://10.120.16.180:48799 restart in Job 52352592. This can be due to memory issue.
Worker tcp://10.120.16.180:51283 restart in Job 52352556. This can be due to memory issue.
Worker tcp://10.120.16.180:56880 restart in Job 52352555. This can be due to memory issue.
Wo

## Kymographs

### Test Parameters

#### Initialize the interactive kymograph class

As a first step, initialize the `tr.interactive.kymograph_interactive` class that will be handling all steps of generating a kymograph. 

You will need to specify the following `args` and `kwargs` (in order):


**Args**

**input_file_prefix (string)** : File prefix for all input hdf5 files of the form "\[input_file_prefix\]\[number\].hdf5" This should be the default output format for the hdf5 export code, but you will need to rename files if taking input files from a different source.

**all_channels (list)** : list of strings corresponding to the different image channels available in the input hdf5 file, with the channel used for segmenting trenches in the first position. NOTE: these names must match those of the input hdf5 file datasets.

**fov_list (list)** : List of ints corresponding to the fovs that you wish to make kymographs of.

**Kwargs**

**t_subsample_step (int)** : Step size to be used for subsampling input files in time, recommend that subsampling results in between 5 and 20 timepoints for quick processing.

**t_range (tuple of ints)** : Range size to be used for subsampling input files in time.

The last line will perform import and subsampling of the input hdf5 image files.

In [6]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interactive_kymograph = tr.interactive.kymograph_interactive(headpath)
channels,fov_list,timepoints_len = interactive_kymograph.get_image_params()

/home/bj66/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/bj66/TrenchRipper/trenchripper/utils.py:91: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.metadata = store.get_storer(key).attrs.metadata


In [7]:
interact(interactive_kymograph.view_image,fov_idx=IntText(value=0,description='FOV number:',disabled=False),\
         t=IntSlider(value=0, min=0, max=timepoints_len-1, step=1,continuous_update=False),
        channel=Dropdown(options=channels,value=channels[0],description='Channel:',disabled=False));

interactive(children=(IntText(value=0, description='FOV number:'), IntSlider(value=0, continuous_update=False,…

<i> Could this be made more robust? E.g. edge detection </i>

In [8]:
import_hdf5 = interactive(interactive_kymograph.import_hdf5_files, {"manual":True},all_channels=fixed(channels),seg_channel=Dropdown(options=channels,value=channels[0]),\
                          fov_list=SelectMultiple(options=fov_list),t_range=IntRangeSlider(value=[0, timepoints_len-1],min=0,max=timepoints_len-1,step=1,disabled=False,continuous_update=False),\
                          t_subsample_step=IntSlider(value=10, min=0, max=200, step=1));
display(import_hdf5)

interactive(children=(Dropdown(description='seg_channel', options=('Phase-Fluor', 'YFP-DUAL'), value='Phase-Fl…

In [9]:
imported_array_list = copy.copy(import_hdf5.result)

#### Tune "trench-row" detection hyperparameters

The kymograph code begins by detecting the positions of trench rows in the image as follows:

1. Reducing each 2D image to a 1D signal along the y-axis by computing the qth percentile of the data along the x-axis
2. Smooth this signal using a median kernel
3. Use a [triangle threshold](https://imagej.net/Auto_Threshold#Triangle) to determine the trench row poisitons

This method uses the following `kwargs`, which you can tune here:

**y_percentile (int)** : Percentile to use for step 1.

**smoothing_kernel_y_dim_0 (int)** : Median kernel size to use for step 2.

**triangle_nbins (int)** : Number of bins to use in the triangle method histogram.

**triangle_scaling (float)** : Scaling factor to apply to the threshold determined by the triangle method.


Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold value for each fov will be displayed as a red line.

In [13]:
matplotlib.rcParams['figure.figsize'] = [20, 10]

row_detection = interactive(interactive_kymograph.preview_y_precentiles, {"manual":True},imported_array_list=fixed(imported_array_list),y_percentile=IntSlider(value=100, min=0, max=100, step=1),\
         smoothing_kernel_y_dim_0=IntSlider(value=17, min=1, max=200, step=2),triangle_nbins=IntSlider(value=50, min=10, max=300, step=10),\
                triangle_scaling=FloatSlider(value=3.5, min=0., max=4., step=0.05),\
            triangle_threshold_bounds=FloatRangeSlider(value=[0, 1],min=0,max=1,step=0.01,disabled=False,continuous_update=False))
display(row_detection)

interactive(children=(IntSlider(value=100, description='y_percentile'), IntSlider(value=17, description='smoot…

#### Generate "trench-row" detection output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps. **Note: The thresholding parameters do not need to be specified at this point.**

In [14]:
y_percentiles_smoothed_list = copy.copy(row_detection.result)

In [15]:
y_percentiles_smoothed_list[0].shape

(2960, 3)

#### Tune "trench-row" cropping hyperparameters

Next, we will use the detected rows to perform cropping of the input image in the y-dimension:

1. Determine edges of trench rows based on threshold mask.
2. Filter out rows that are too small.
3. Perform cropping using the "end" of the row as reference (the end referring to the part of the trench farthest from the feeding channel).

This method uses the following `kwargs`, which you can tune here:

**y_min_edge_dist (int)** : Minimum row length necessary for detection.

**padding_y (int)** : Padding to be used when cropping in the y-dimension.

**trench_len_y (int)** : Length from the end of the tenches to be used when cropping in the y-dimension. <i> why can't this just be found using the edge nearest to the feeding channel? </i>

**top_orientation (int)** : The orientation of the top-most row where 0 corresponds to a trench with a downward-oriented trench opening and 1 corresponds to a trench with an upward-oriented trench opening.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display y-cropped images for each fov and timepoint.

In [16]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
y_cropping = interactive(interactive_kymograph.preview_y_crop,{"manual":True},y_percentiles_smoothed_list=fixed(y_percentiles_smoothed_list),\
                imported_array_list=fixed(imported_array_list),\
                y_min_edge_dist=IntSlider(value=50, min=10, max=200, step=10),\
                padding_y=IntSlider(value=20, min=0, max=100, step=1),\
                trench_len_y=IntText(value=270, description="Trench Length (y)"),
               vertical_spacing=FloatSlider(value=0.9, min=0., max=2., step=0.01),\
                expected_num_rows=IntText(value=2,description='Number of Rows:',disabled=False),\
               orientation_detection=Dropdown(options=[0, 1, 'phase'],value=0,description='Orientation:',disabled=False),
                orientation_on_fail=Dropdown(options=[None,0, 1],value=0,description='Orientation when < expected rows:',disabled=False))
display(y_cropping)

interactive(children=(IntSlider(value=50, description='y_min_edge_dist', max=200, min=10, step=10), IntSlider(…

#### Generate "trench-row" cropping output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps.

In [13]:
cropped_in_y_list = copy.copy(y_cropping.result)

#### Tune trench detection hyperparameters

Next, we will detect the positions of trenchs in the y-cropped images as follows:

1. Reducing each 2D image to a 1D signal along the x-axis by computing the qth percentile of the data along the y-axis.
2. Determine the signal background by smooth this signal using a large median kernel.
3. Subtract the background signal.
4. Smooth the resultant signal using a median kernel.
5. Use a [otsu threhsold](https://imagej.net/Auto_Threshold#Otsu) to determine the trench midpoint poisitons.

This method uses the following `kwargs`, which you can tune here:

**x_percentile (int)** : Percentile to use for step 1.

**background_kernel_x (int)** : Median kernel size to use for step 2.

**smoothing_kernel_x (int)** : Median kernel size to use for step 4.

**otsu_nbins (int)** : Number of bins to use in the Otsu's method histogram.

**otsu_scaling (float)** : Scaling factor to apply to the threshold determined by Otsu's method.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold value for each fov will be displayed as a red line.

In [14]:
trench_detection = interactive(interactive_kymograph.preview_x_percentiles, {"manual":True}, cropped_in_y_list=fixed(cropped_in_y_list),t=IntSlider(value=0, min=0, max=cropped_in_y_list[0].shape[4]-1, step=1),\
                x_percentile=IntSlider(value=85, min=50, max=100, step=1),background_kernel_x=IntSlider(value=21, min=1, max=601, step=20), smoothing_kernel_x=IntSlider(value=9, min=1, max=31, step=2),\
               otsu_nbins=IntSlider(value=50, min=10, max=200, step=10),otsu_scaling=FloatSlider(value=0.25, min=0., max=2., step=0.01),\
               vertical_spacing=FloatSlider(value=0.9, min=0., max=2., step=0.01));
display(trench_detection)

interactive(children=(IntSlider(value=0, description='t', max=1), IntSlider(value=85, description='x_percentil…

#### Generate trench detection output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps. **Note: The thresholding parameters do not need to be specified at this point.**

In [15]:
smoothed_x_percentiles_list = trench_detection.result

#### Check midpoint drift

Next, we will perform x-dimension drift correction of our detected midpoints as follows:

1. Begin at t=1
2. For $m \in \{midpoints(t)\}$ assign $n \in \{midpoints(t-1)\}$ to m if n is the closest midpoint to m at time $t-1$,
points that are not the closest midpoint to any midpoints in m will not be mapped.
3. Compute the translation of each midpoint at time.
4. Take the average of this value as the x-dimension drift from time t-1 to t.

This method uses the following `kwargs`, which you can tune here:

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display the detected midpoints for each of your timepoints. If there is too much sparsity, or discontinuity, your drift correction will not be accurate.

<i> Number/trench interest point registration (SIFT)? Autocorrelation? Single-dimension Euclidean distances between intensity maps of trench-sized boxes?</i>

In [16]:
midpoint_drift = interactive(interactive_kymograph.preview_midpoints,{"manual":True},smoothed_x_percentiles_list=fixed(smoothed_x_percentiles_list),\
               vertical_spacing=FloatSlider(value=0.8, min=0., max=2., step=0.01));
display(midpoint_drift)

interactive(children=(FloatSlider(value=0.8, description='vertical_spacing', max=2.0, step=0.01), Button(descr…

#### Generate midpoint drift output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps.

In [17]:
all_midpoints_list,x_drift_list = midpoint_drift.result

#### Tune trench cropping hyperparameters

Trench cropping simply uses the drift-corrected midpoints as a reference and crops out some fixed length around them to produce an output kymograph

This method uses the following `kwargs`, which you can tune here:

**trench_width_x (int)** : Trench width to use for cropping.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display a random kymograph for each row in each fov.

It will also produce midpoint plots showing retained midpoints

In [18]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interact_manual(interactive_kymograph.preview_kymographs,cropped_in_y_list=fixed(cropped_in_y_list),all_midpoints_list=fixed(all_midpoints_list),\
                x_drift_list=fixed(x_drift_list),trench_width_x=IntSlider(value=30, min=10, max=100, step=2),\
                trench_present_thr=FloatSlider(value=0., min=0., max=1., step=0.05),\
               vertical_spacing=FloatSlider(value=0.8, min=0., max=2., step=0.01))

interactive(children=(IntSlider(value=30, description='trench_width_x', min=10, step=2), FloatSlider(value=0.0…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

<i> Why do we store data as kymographs? </i>

#### Export and save hyperparameters

In [19]:
interactive_kymograph.process_results()

Y Percentile 80
Y Smoothing Kernel 17
Triangle Threshold Bins 90
Triangle Threshold Scaling 1.35
Minimum Trench Length 50
Y Padding 20
Trench Length 700
Orientation Detection Method 0
Expected Number of Rows (Manual Orientation Detection) 1
Top Orientation when Row Drifts Out (Manual Orientation Detection) 0
X Percentile 85
X Background Kernel 221
X Smoothing Kernel 17
Otsu Threshold Bins 50
Otsu Threshold Scaling 0.31
Trench Width 50
Trench Presence Threshold 0.0
All Channels ['Phase']
Time Range (0, 15)


In [20]:
interactive_kymograph.write_param_file()

### Generate Kymograph

#### Start Dask Workers

In [21]:
dask_controller = tr.cluster.dask_controller(walltime='04:00:00',local=False,n_workers=20,memory='4GB')
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

/home/bj66/anaconda3/lib/python3.7/site-packages/bokeh/plotting/helpers.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, OrderedDict, Sequence


In [22]:
dask_controller.displaydashboard()

In [23]:
kymoclust = tr.kymograph.kymograph_cluster(headpath=headpath,trenches_per_file=20,paramfile=True)

In [24]:
kymoclust.generate_kymographs(dask_controller)

/home/bj66/TrenchRipper/trenchripper/utils.py:91: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.metadata = store.get_storer(key).attrs.metadata
/home/bj66/anaconda3/lib/python3.7/site-packages/bokeh/core/property/container.py:103: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(value, (collections.Container, collections.Sized, collections.Iterable))


In [ ]:
kymoclust.post_process(dask_controller)

#### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

#### Shutdown Dask

In [81]:
dask_controller.shutdown()

Worker tcp://10.120.16.192:49717 restart in Job 52359202. This can be due to memory issue.
Worker tcp://10.120.17.13:50000 restart in Job 52359189. This can be due to memory issue.
Worker tcp://10.120.16.192:53878 restart in Job 52359199. This can be due to memory issue.
Worker tcp://10.120.17.13:43041 restart in Job 52359191. This can be due to memory issue.
Worker tcp://10.120.16.192:36391 restart in Job 52359201. This can be due to memory issue.
Worker tcp://10.120.17.13:38638 restart in Job 52359190. This can be due to memory issue.
Worker tcp://10.120.16.192:47320 restart in Job 52359200. This can be due to memory issue.
Worker tcp://10.120.16.191:60985 restart in Job 52359207. This can be due to memory issue.
Worker tcp://10.120.16.191:50421 restart in Job 52359208. This can be due to memory issue.
Worker tcp://10.120.16.191:56622 restart in Job 52359205. This can be due to memory issue.
Worker tcp://10.120.16.191:56936 restart in Job 52359206. This can be due to memory issue.
Wo

## Fluorescence Segmentation

### Test Parameters

#### Initialize the interactive segmentation class

As a first step, initialize the `tr.interactive.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

You will need to specify the following `args` (in order):


**Args**

**headpath (string)** : Top level path being used for processing (same as the rest of the mother machine pipeline.)

In [ ]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interactive_segmentation = tr.interactive.fluo_segmentation_interactive(headpath)

#### Choose channel to segment on

In [ ]:
choose_channel = interactive(interactive_segmentation.choose_seg_channel, {"manual":True},seg_channel=Dropdown(options=interactive_segmentation.all_channels,\
                                                                                              value=interactive_segmentation.all_channels[0]));
display(choose_channel)

#### Import data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**fov_idx (int)** :

**n_trenches (int)** :

**t_range (tuple)** :

**t_subsample_step (int)** :

In [ ]:
kymo_arr_int = interactive(interactive_segmentation.import_array, {"manual":True},\
                       n_trenches=IntText(value=1,description='Number of trenches:',disabled=False),\
                       t_range=IntRangeSlider(value=[interactive_segmentation.t_range[0],interactive_segmentation.t_range[1]-1],\
                        description='Time Range:',min=interactive_segmentation.t_range[0],max=interactive_segmentation.t_range[1]-1,step=1,disabled=False),\
                       t_subsample_step=IntSlider(value=1,description='Time Subsampling Step:', min=1, max=20, step=1),\
                       fig_size_y=IntSlider(value=9, description='Figure Size (Y Dimension):' , min=1, max=30, step=1),\
                       fig_size_x=IntSlider(value=6, description='Figure Size (X Dimension):', min=1, max=30, step=1),\
                          img_per_row=IntSlider(value=2, description='Images per Row:', min=1, max=30, step=1));
display(kymo_arr_int)

In [ ]:
kymo_arr = copy.copy(kymo_arr_int.result)

#### Scale data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**scale (bool)** : Whether to scale the kymograph in time.

**scaling_percentile (int)** : Whole image intensity percentile to use to determine scaling constant. 

In [ ]:
scaled_list_int = interactive(interactive_segmentation.plot_scaled, {"manual":True},kymo_arr=fixed(kymo_arr),\
                          scale=Dropdown(options=[True,False],value=True,description='Scale Fluorescence?',disabled=False),\
                          scaling_percentile=IntSlider(value=90,description='Scaling Percentile:',min=0,max=100,step=1,disabled=False));
display(scaled_list_int)

In [ ]:
scaled_list = copy.copy(scaled_list_int.result)

#### Apply Gaussian Filter

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**smooth_sigma (float)** : Standard deviation of gaussian kernel.

In [ ]:
proc_list_int = interactive(interactive_segmentation.plot_processed, {"manual":True},scaled_list=fixed(scaled_list),\
                          smooth_sigma=FloatSlider(value=0.75,description='Gaussian Kernel Sigma:',min=0.,max=3.,step=0.25,disabled=False));
display(proc_list_int)

In [ ]:
proc_list = copy.copy(proc_list_int.result)
eigval_list = interactive_segmentation.plot_eigval(proc_list)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(eigval_list[3]>125)

#### Determine Cell Mask Envelope

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**cell_mask_method (str)** : Thresholding method, can be a local or global Otsu threshold.

**cell_otsu_scaling (float)** : Scaling factor applied to determined threshold.

**local_otsu_r (int)** : Radius of thresholding kernel used in the local otsu thresholding.

In [ ]:
cell_mask_list_int = interactive(interactive_segmentation.plot_cell_mask, {"manual":True},proc_list=fixed(proc_list),\
                          cell_mask_method=Dropdown(options=['local','global'],value='local',description='Cell Mask Thresholding Method:',disabled=False),\
                          global_otsu_scaling=FloatSlider(value=0.95,description='Global Threshold Scaling:',min=0.,max=2.,step=0.01,disabled=False),\
                          cell_otsu_scaling=FloatSlider(value=0.95,description='Cell Threshold Scaling:',min=0.,max=2.,step=0.01,disabled=False),\
                          local_otsu_r=IntSlider(value=15,description='Local Otsu Radius:',min=0,max=30,step=1,disabled=False));
display(cell_mask_list_int)

In [ ]:
cell_mask_list = copy.copy(cell_mask_list_int.result)

#### Display Edge Mask at Threshold Value

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**edge_threshold_scaling (float)** : Scaling factor applied to determined threshold.

In [ ]:
composite_mask_list_int = interactive(interactive_segmentation.plot_threshold_result, {"manual":True},eigval_list=fixed(eigval_list),cell_mask_list=fixed(cell_mask_list),\
                          edge_threshold_scaling=FloatSlider(value=1.,description='Edge Threshold Scaling',min=0.,max=2.,step=0.01,disabled=False),\
                          min_obj_size=IntSlider(value=30,description='Minimum Object Size:',min=0,max=100,step=2,disabled=False));
display(composite_mask_list_int)

#### Threshold Sampling and Convexity Calculation

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**edge_threshold_scaling (float)** : Scaling factor applied to determined threshold.

**threshold_step_perc (float)** : Threshold step size to be used for trying multiple thresholds.

**threshold_perc_num_steps (int)** : Number of steps to use when generating multiple thresholds.

In [ ]:
conv_scores_list_int = interactive(interactive_segmentation.plot_scores, {"manual":True},eigval_list=fixed(eigval_list),cell_mask_list=fixed(cell_mask_list),\
                          edge_threshold_scaling=FloatSlider(value=0.9,description='Edge Threshold Scaling',min=0.,max=2.,step=0.01,disabled=False),\
                          threshold_step_perc=FloatSlider(value=0.05,description='Threshold Step Percent',min=0.,max=0.5,step=0.01,disabled=False),\
                          threshold_perc_num_steps=IntSlider(value=2,description='Number of Threshold Steps',min=0,max=5,step=1,disabled=False),\
                          min_obj_size=IntSlider(value=30,description='Minimum Object Size:',min=0,max=100,step=2,disabled=False));
display(conv_scores_list_int)

In [ ]:
conv_scores_list = copy.copy(conv_scores_list_int.result)

#### Convexity Thresholding

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**convex_threshold (float)** : Threshold to be used for convexity thresholding.

In [ ]:
final_mask_list_int = interactive(interactive_segmentation.plot_final_mask, {"manual":True},conv_scores_list=fixed(conv_scores_list),\
                          convex_threshold=FloatSlider(value=0.75,description='Convexity Threshold:',min=0.,max=1.,step=0.01,disabled=False));
display(final_mask_list_int)

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.cluster.dask_controller(walltime='01:00:00',local=False,n_workers=100,memory='500MB')
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath,paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

## Phase Segmentation Training

### Data Preparation

In [ ]:
dataloader = tr.unet.UNet_Training_DataLoader(nndatapath="/n/scratch2/de64/nntest5",experimentname="First NN",trainpath="/n/scratch2/de64/2019-06-18_DE85_training_data",\
                                      testpath="/n/scratch2/de64/2019-05-31_validation_data",\
                                      valpath="/n/scratch2/de64/2019-05-31_validation_data")

#### Training Set Selection

In [ ]:
dataloader.inter_get_selection(dataloader.trainpath,"train")

#### Test Set Selection

In [ ]:
dataloader.inter_get_selection(dataloader.testpath,"test")

#### Validation Set Selection

In [ ]:
dataloader.inter_get_selection(dataloader.valpath,"val")

#### Weightmap Parameters

In [ ]:
dataloader.display_grid()

In [ ]:
dataloader.get_grid_params()

#### Export

In [ ]:
dataloader.export_all_data(memory="6GB")

### Hyperparameter (Grid) Search

#### Set-up Search

In [ ]:
grid = tr.unet.GridSearch("/n/scratch2/de64/nntest4",numepochs=50)

In [ ]:
grid.display_grid()

In [ ]:
grid.get_grid_params()

#### Run Search

In [ ]:
grid.run_grid_search()

## Other

#### Dask Utilities

In [ ]:
dask_controller.shutdown()

In [ ]:
dask_controller.retry_failed()

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.retry_processing()